In [ ]:
pyspark --master=local[1] --jars /root/library/commons-httpclient-3.0.1.jar,/root/library/spark-sql-kafka-0-10_2.12-3.1.1.jar,/root/library/commons-pool2-2.10.0.jar,/root/library/kafka-clients-2.8.0.jar,/root/library/spark-token-provider-kafka-0-10_2.12-3.1.1.jar

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark import SparkContext
from pyspark.conf import SparkConf


In [ ]:
sc.addFile("/root/mymodel/mymodel.py")
import mymodel

In [ ]:
df_raw = spark \
  .readStream \
  .format('kafka') \
  .option('kafka.bootstrap.servers', 'localhost:9092') \
  .option("startingOffsets", "earliest") \
  .option('subscribe', 'inchat') \
  .load()


In [ ]:
json_data = df_raw.selectExpr('CAST(value AS STRING) as value')

In [ ]:
schema = StructType([ \
    StructField("video_id",StringType(),True), \
    StructField("user_name",StringType(),True), \
    StructField("chat_id",StringType(),True), \
    StructField("chat_text",StringType(),True), \
    StructField("h_token", ArrayType(StringType()),True), \
    StructField("modified_text",StringType(),True), \
    StructField("noun_token",ArrayType(StringType(),True)), \
    StructField("chat_time", TimestampType(),True)
  ])


In [ ]:
df_data = json_data.select(from_json(col("value"), schema).alias('data'))

In [ ]:
df_data.withColumn("emotion",lit(mymodel.sentiment_predict("data.h_token"))) \
.selectExpr("CAST(data AS STRING) AS key", "to_json(struct(*)) AS value") \
.writeStream.trigger(processingTime='2 seconds') \
.format('kafka') \
.option('kafka.bootstrap.servers', 'localhost:9092') \
.option('topic', 'outchat') \
.option("truncate", False).option("checkpointLocation", "/tmp/dtn/checkpoint") \
.start().awaitTermination()